In [2]:
import sys
sys.path.append('../src')
from data import get_documents_list

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


loaded spacy language model: en_core_web_sm


https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models

In [20]:
embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
#Usage: 

embeddings = embedder.encode(sentences)

#Storing:
    
import pickle

Store sentences & embeddings on disc

with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': sentences, 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

Load sentences & embeddings from disc

with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [15]:
from data import get_documents_list, get_cleaned_dataframe_with_topics
from models import classification, get_embedding_model, get_BERTopic_model

df = get_cleaned_dataframe_with_topics()
df['cleaned_texts_embeddings'] = df['cleaned_texts'].apply(embedder.encode)

In [28]:
df.head() # überall doppelte Klammern!

,cleaned_texts,cleaned_paragraphs,topics,topics_lists,cleaned_texts_topics,cleaned_paragraphs_topics,cleaned_sentences,label,cleaned_texts_embeddings
0,"[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septemb...","[[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septem...","[prize, mozart, mendelssohn, awarded, mozarteu...","[[prize, mozart, mendelssohn, awarded, mozarte...","[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septemb...","[[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septem...","[[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septem...",0,"[[0.013540071, -0.02685532, -0.0390787, -0.029..."
1,"[Alexander, Zemlinsky, Alexander, von, Zemlins...","[[Alexander, Zemlinsky, Alexander, von, Zemlin...","[webern, schoenberg, schoenbergs, weberns, lei...","[[webern, schoenberg, schoenbergs, weberns, le...","[Alexander, Zemlinsky, Alexander, von, Zemlins...","[[Alexander, Zemlinsky, Alexander, von, Zemlin...","[[Alexander, Zemlinsky, Alexander, von, Zemlin...",0,"[[-0.0028326442, -0.0070843427, -0.044704553, ..."
2,"[Alexander, Tansman, Polish, Aleksander, Tansm...","[[Alexander, Tansman, Polish, Aleksander, Tans...","[webern, schoenberg, schoenbergs, weberns, lei...","[[webern, schoenberg, schoenbergs, weberns, le...","[Alexander, Tansman, Polish, Aleksander, Tansm...","[[Alexander, Tansman, Polish, Aleksander, Tans...","[[Alexander, Tansman, Polish, Aleksander, Tans...",0,"[[-0.0028326442, -0.0070843427, -0.044704553, ..."
3,"[Alfred, Szendrei, also, Alfred, Sendrey, Alad...","[[Alfred, Szendrei, also, Alfred, Sendrey, Ala...","[liszt, academy, zoltn, franz, kodly, bla, hun...","[[liszt, academy, zoltn, franz, kodly, bla, hu...","[Alfred, Szendrei, also, Alfred, Sendrey, Alad...","[[Alfred, Szendrei, also, Alfred, Sendrey, Ala...","[[Alfred, Szendrei, also, Alfred, Sendrey, Ala...",0,"[[0.04835217, -0.049049504, -0.035721995, -0.0..."
4,"[Andre, Asriel, 22, February, 1922, 28, May, 2...","[[Andre, Asriel, 22, February, 1922, 28, May, ...","[prize, mozart, mendelssohn, awarded, mozarteu...","[[prize, mozart, mendelssohn, awarded, mozarte...","[Andre, Asriel, 22, February, 1922, 28, May, 2...","[[Andre, Asriel, 22, February, 1922, 28, May, ...","[[Andre, Asriel, 22, February, 1922, 28, May, ...",0,"[[0.012281087, -0.020743437, -0.04728438, -0.0..."


In [23]:
X = df['cleaned_texts_embeddings'].to_list()
y = df['label'].to_list()

In [29]:
X

[array([[-0.00862336,  0.0160649 , -0.01440935, ...,  0.00286973,
          0.01759599, -0.01537712],
        [-0.03647278,  0.01151731, -0.037663  , ..., -0.01218475,
          0.05712207, -0.02305481],
        [ 0.0567284 , -0.06898203, -0.03820982, ..., -0.07615896,
          0.02582713, -0.04186673],
        ...,
        [ 0.006207  ,  0.01313599, -0.07654863, ..., -0.03089185,
         -0.01506457, -0.04718573],
        [-0.00524695, -0.02658997, -0.06650022, ...,  0.01094562,
          0.02290443, -0.02823039],
        [ 0.02423782, -0.02501504,  0.01598188, ..., -0.10043467,
          0.03865677, -0.0441542 ]], dtype=float32),
 array([[ 0.06976597, -0.01622735,  0.01184594, ..., -0.01242758,
          0.0032396 ,  0.04087415],
        [-0.00011255, -0.00709976, -0.03879236, ..., -0.05591784,
          0.00244486,  0.02164791],
        [ 0.0645923 , -0.04696368, -0.02985672, ..., -0.07866296,
         -0.00708421, -0.05861729],
        ...,
        [ 0.03351326, -0.04190839, -0.0

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [27]:
y_pred = classification.classify_predict(X_train_vect_avg, X_test_vect_avg, y_train, "Random Forest")
classification.print_classification_report(y_test, y_pred)

y_pred = classification.classify_predict(X_train_vect_avg, X_test_vect_avg, y_train, "Multinomial Naive Bayes")
classification.print_classification_report(y_test, y_pred)

y_pred = classification.classify_predict(X_train_vect_avg, X_test_vect_avg, y_train, "SVM")
classification.print_classification_report(y_test, y_pred)

############################################################################
Processing Random Forest Classification



AttributeError: 'list' object has no attribute 'values'